# EJERCICIO DE DIFUSIÓN DE INFORMACIÓN
## Redes Sociales, Colaboración en Red

**Autores:**  
Martín Salinas Antón - martin.salinas@estudiante.uam.es  
Belén Vivas García - belen.vivas@estudiante.uam.es

En esta práctica intentaremos buscar otra forma de decidir cuáles son los nodos (personajes) más importantes y que más influencia tienen sobre la red.  
  
Las medidas de centralidad son rápidas de calcular y los resultados suelen ser bastante correctos, pero están más orientadas al número de nodos que haya o el número de conexiones entre ellos, y tal vez dejen atrás otros factores y no sean tan efectivas a la hora de estimar cuáles son los nodos que difunden mejor la información.  
  
Para la difusión de la información, es decir, encontrar los personajes que mejor puedan propagar la información, posiblemente sea mejor utilizar algoritmos de **cascada** que tengan en cuenta las **probabilidades de activación** de los nodos en sus conexiones con otros nodos. Por eso, para esta práctica implementaremos el algoritmo codicioso para maximizar la influencia de la red, y ver cuáles son los nodos que, activándolos desde un principio, puedan maximizar esta influencia. Estos nodos serán la **semilla**.  
  
La difusión de la información mediante este método tiene un inconveniente, y es el tiempo de ejecución, el cual puede ser bastante alto dependiendo del presupuesto (k) que introduzcamos. No existe algoritmo óptimo, ya que se trata de un problema NP-hard. También, al utilizar el algoritmo codicioso, podemos obtener, al menos, el 63% del valor óptimo, es decir, que no es tan estable como las medidas de centralidad.

In [1]:
import networkx as nx
import random
import numpy as np
import time

### Lectura de Datos

En primer lugar, leemos el grafo con NetworkX.

In [2]:
G = nx.read_graphml('data/juegoDtronos.graphml')

### Normalización de Pesos

El primer paso será normalizar los pesos de los vértices del grafo entre 0 y 1 para que el algoritmo los tome como probabilidades. Estas serán las probabilidades de activación de los nodos.

In [3]:
# Normalize weights into probabilities
weights = []
for edge in G.edges(data=True):
    # Get weight from graph and add it to the weight list
    weight = edge[2]['weight']
    weights.append(weight)

normalized_weights = []
# Get the maximum weight
max_weight = max(weights)
for w in weights:
    # Divide each weight by the maximum to normalize between 0 and 1
    w /= max_weight
    normalized_weights.append(w)

# Get a copy of the graph
G_norm = G
# Modify the weights
i = 0
for edge in G_norm.edges(data=True):
    edge[2]['weight'] = normalized_weights[i]
    i += 1

*Nota: no hacemos la conversión explícita a grafo no dirigido ya que, en las ejecuciones que hemos realizado abajo, los resultados salían iguales con ambos tipos de grafo.*

### Algoritmo Codicioso y de Cascada

Implementamos los algoritmos necesarios para maximizar la influencia de la red. La influencia de la red se maximiza seleccionando un grupo de nodos, a los que llamaremos 'semilla', los cuales tienen una influencia suficiente para maximizar la propagación de información.  
  
La aproximación codiciosa (o greedy) selecciona los nodos con mayor influencia en función de la cantidad de vecinos, utilizando una función f que, en nuestro caso, selecciona el número de nodos que serán finalmente activados teniendo en cuenta los pesos (ahora probabilidades) de los vértices de los nodos. Esta función f devolverá el número de nodos que se activarán en cada iteración.

In [4]:
# Maximizing the spread of cascades - greedy algorithm
def max_spread_cascades_greedy(G, k):

    # Set of initially activated nodes
    S = set()

    # While i =/= k
    for _ in range(k):
        v = None
        max_f = -1

        for node in G.nodes():
            if node not in S:
                # Cascade function taking S U {v}
                f = cascade(G, S.union({node}))

                # Update maximized f
                if f > max_f:
                    v = node
                    max_f = f

        # Add chosen node to the set
        # S U {v}
        S.add(v)

    return S


# Gets the number of ultimately activated nodes
def cascade(G, S):

    # Set of activated nodes
    A = S
    # Set of neighbors of nodes in S
    activable_neighbors = set()

    # Add neighbors to the set
    for node in S:
        activable_neighbors.update(G.neighbors(node))

    # While there are still activable neighbors
    while activable_neighbors:
        # Choose random neighbor
        node = random.choice(list(activable_neighbors))

        # Activation probability of the node
        probability = 1.0
        for n in G.neighbors(node):
            if n not in A:
                if n not in activable_neighbors:
                    # Take the node weight (now a probability)
                    weight = G[node][n]['weight']
                    probability *= (1 - weight)

        # Activation
        if np.random.rand() < probability:
            # Add activated node
            A.add(node)
            # Add neighbors of activated node
            for n in G.neighbors(node):
                if n not in A:
                    activable_neighbors.add(n)

        # Update neighbors
        activable_neighbors.remove(node)

    return len(A)

Establecemos k (budget) y ejecutamos el algoritmo para obtener la semilla.  
  
Realizaremos varias ejecuciones variando k para ver los resultados obtenidos y, además, mediremos el tiempo de cada ejecución para tener una noción de lo costoso que es este método.

In [5]:
# Print results for each execution
def print_results(seed, k, time):
    print('Budget (k) = {}\n------------------'.format(k))
    for s in seed:
        print(s)
    print('-------------------\nExecution time: {:.2f}s'.format(time))

In [6]:
# Budget: 3
k = 3
start_time = time.time()
seed = max_spread_cascades_greedy(G_norm, k)
end_time = time.time()
print_results(seed, k, end_time - start_time)


Budget (k) = 3
------------------
Jalabhar-Xho
Addam-Marbrand
Cersei-Lannister
-------------------
Execution time: 9.58s


In [7]:
# Budget: 8
k = 8
start_time = time.time()
seed = max_spread_cascades_greedy(G_norm, k)
end_time = time.time()
print_results(seed, k, end_time - start_time)

Budget (k) = 8
------------------
Brynden-Tully
Gyles-Rosby
Jalabhar-Xho
Joffrey-Baratheon
Cersei-Lannister
Oberyn-Martell
Jaime-Lannister
Addam-Marbrand
-------------------
Execution time: 27.16s


In [8]:
# Budget: 15
k = 15
start_time = time.time()
seed = max_spread_cascades_greedy(G_norm, k)
end_time = time.time()
print_results(seed, k, end_time - start_time)

Budget (k) = 15
------------------
Robb-Stark
Varys
Tywin-Lannister
Kevan-Lannister
Lyle-Crakehall
Brynden-Tully
Gyles-Rosby
Jalabhar-Xho
Tyrion-Lannister
Joffrey-Baratheon
Cersei-Lannister
Oberyn-Martell
Catelyn-Stark
Jaime-Lannister
Addam-Marbrand
-------------------
Execution time: 52.27s


In [9]:
# Budget: 20
k = 20
start_time = time.time()
seed = max_spread_cascades_greedy(G_norm, k)
end_time = time.time()
print_results(seed, k, end_time - start_time)

Budget (k) = 20
------------------
Tywin-Lannister
Theon-Greyjoy
Cersei-Lannister
Catelyn-Stark
Varys
Tyrion-Lannister
Robb-Stark
Brynden-Tully
Rickard-Karstark
Arya-Stark
Oberyn-Martell
Jaime-Lannister
Edmure-Tully
Walder-Frey
Kevan-Lannister
Lyle-Crakehall
Gyles-Rosby
Joffrey-Baratheon
Jalabhar-Xho
Addam-Marbrand
-------------------
Execution time: 70.27s


En cada ejecución obtenemos una semilla del tamaño del presupuesto establecido.  
  
Con k=3, la semilla obtenida no corresponde a ningún nodo influenciable obtenido con las medidas de centralidad, por lo que subiremos el presupuesto. El tiempo de ejecución es, además, relativamente bajo, de 9.58s. Es visiblemente más costoso que las medidas de centralidad, y no tiene mucha efectividad.  
  
Con k=8, ya encontramos nodos más influyentes como Joffrey Baratheon, Cersei Lannister o Jaime Lannister. Tarda más en ejecutarse, 27.16s.  
  
Probando con k=15, ya vemos resultados más "esperables". Es lógico, teniendo en cuenta que el grafo es muy grande, por lo que necesitaremos un presupuesto relativamente grande. Encontramos nodos que ya encontrábamos con las medidas de centralidad, y nodos que, en la serie, se consideran bastante influyentes y grandes difusores de información, como Varys, Tywin Lannister en las primeras temporadas, Tyrion Lannister u Oberyn Martell. Tarda 52.27s en ejecutarse, empezando a ser ya un tiempo considerable.  
  
Finalmente, con k=20 obtenemos una semilla de gran tamaño, pero no se añaden nodos demasiado influyentes a los que ya había con un presupuesto menor, y tarda 70.27s. Probar con más presupuesto dejaría de ser efectivo y subiría demasiado el tiempo de ejecución.  
  
Por tanto, **k=15** sería una semilla buena, obteniendo los personajes que maximizan la influencia de la red. No todos los personajes son los mismos que los obtenidos con medidas de centralidad como de grado o de vector propio, pero este método considera otros factores, como las probabilidades y, además, al ser un algoritmo codicioso, no devuelve resultados ideales. Aún así, nos parecen resultados lo suficientemente buenos para la iniciación de la difusión de información, porque hay personajes muy importantes pero, tal vez, en otros aspectos, y no en este en concreto.  
  
Los personajes que más coinciden con los obtenidos en las medidas de centralidad serían:  
- Tyrion Lannister
- Jaime Lannister
- Cersei Lannister
    
Por tanto, estos personajes serían, además de influyentes en otros aspectos, buenos difusores de información.